In [59]:
# 의사록(MPB)

import requests
from bs4 import BeautifulSoup

for i in range(1, 45):
    b_url = "https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761"
    params = {
        'pageIndex' : i
    }

    response = requests.get(b_url, params=params)
    soup = BeautifulSoup(response.content)

    li_list = []
    li = soup.find('div', class_='bdLine type2').find('ul').find('li')
    li_li = li.find_next_siblings('li', li)
    li_list.append(li)
    li_list.extend(li_li)

    print(i)

    for x in range(0, len(li_list)):
        try:
            link = li_list[x].find('div', class_='fileGoupBox').find('ul').find('li').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url2 = 'http://www.bok.or.kr' + link
            file_res = requests.get(url2)

            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res.content)
        except:
            link2 = li_list[x].find('div').find('div').find('a').attrs['href']
            title = li_list[x].find('div', class_='row').find('span').find('a').find('span').find('span').text
            url3 = 'http://www.bok.or.kr' + link2
            file_res2 = requests.get(url3)

            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(file_res.content)
            



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [57]:
# 채권 분석 리포트 https://finance.naver.com/research/debenture_list.naver
import time
from datetime import datetime
import requests
from bs4 import BeautifulSoup

for i in range (1, 198):
    bond_url = "https://finance.naver.com/research/debenture_list.naver"
    params = {
        'page' : i
    }
    headers = {'authority' : 'finance.naver.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36' }
   

    response = requests.get(bond_url, params=params, headers=headers)
    soup = BeautifulSoup(response.content)
    bond = soup.select('#contentarea_left .file')
    bond_list = []
    for x in range(len(bond)):
        if bond[x].find_previous_sibling('td').text == '유안타증권':
            try:    
                bond_list.append([soup.select("#contentarea_left .file a[href$='.pdf']")[x].attrs['href'], bond[x].find_previous_sibling('td').find_previous_sibling('td').text])
            except:
                print("통과")

    for y in bond_list:
        file_res = requests.get(y[0])
        title = y[-1]
        try:
            with open('{}.pdf'.format(title), 'wb') as f:
                f.write(file_res.content)
        except:
            print('통과')


    
            

통과
통과


1. 월별 기사수 '금리'라는 단어 포함
2. 각 신문사별 기사수
3. 채권분석데이터수

In [6]:
# NNC(NaverNewsCrawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

date_date = pd.date_range('2012-12-30', '2023-08-31', freq='W') #2013년부터 2023년까지 뉴스데이터 크롤링
date_role = [date_date[:130],date_date[130:260],date_date[260:390],date_date[390:520],date_date[520:]]
df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])

for date in tqdm(date_role[0]):  #자신의 번호로 인덱싱
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(6)).strftime('%Y-%m-%d')
    last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(response.content)
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] #마지막 페이지 가져오기
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 마지막 페이지까지 반복문
        news_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(response.content)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press') #신문사 데이터
        newsdate = soup.select('.articleSummary .wdate') # 날짜 데이터

        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))  #amp;제거해서 링크활성화
            magazine_name.append(magazine[i].text)
            dates.append(newsdate[i])

        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.content)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, newsdate[index].text.lstrip()[:10]])
        
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])  #모든 리스트를 데이터프레임으로 전환
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{DateStart}_{DateEnd}.csv', encoding='utf-8') # 1주일 간격으로 크롤링 데이터 저장    

  0%|          | 0/130 [00:00<?, ?it/s]

43


  1%|          | 1/130 [03:53<8:21:09, 233.10s/it]

79


  2%|▏         | 2/130 [09:44<10:45:30, 302.58s/it]

51


https://finance.naver.com/news/news_read.naver?article_id=0000027614&office_id=050&mode=search&query=금리&page=4


In [66]:
# NNC(NaverNewsCrawling)
import time
import datetime
import requests, os
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

date_range = pd.date_range('2013-01-01', '2023-08-31', freq='W')
date_role = date_range[::111]
date_range = pd.date_range('2013-01-01', '2023-08-31', freq='W')
df = pd.DataFrame(columns =['신문사', '제목', '내용', '날짜'])


for date in tqdm(date_range):
    DateStart = date.strftime('%Y-%m-%d')
    DateEnd = (date+datetime.timedelta(7)).strftime('%Y-%m-%d')
    last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
    headers = {'authority' : 'finance.naver.com',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
    response = requests.get(last_url, headers=headers)
    soup = BeautifulSoup(response.content)
    last_page = soup.select('.pgRR a')[0]['href'].split('=')[-1] # =기준 split으로 수정
    print(last_page)
    for i in tqdm(range(1, int(last_page)+1)):   # 여기서 다양한 최종페이지까지 설정
        news_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page={i}"
        headers = {'authority' : 'finance.naver.com',
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
        response = requests.get(news_url, headers=headers)
        soup = BeautifulSoup(response.content)
        news = soup.select('.thumb a' )
        magazine = soup.select('.articleSummary .press')
        date = soup.select('.articleSummary .wdate')

        news_link = []
        magazine_name = []
        dates = []
        for i in range(len(news)):
            news_link.append('https://finance.naver.com' + news[i].attrs['href'].replace('amp;', ''))
            magazine_name.append(magazine[i].text)
            dates.append(date[i])

        total = []
        for index, j in enumerate(news_link):
            news_resp = requests.get(j, headers=headers)
            news_soup = BeautifulSoup(news_resp.content)
            try:
                title = news_soup.select('.article_info h3')[0].text.strip()
                content = news_soup.select('#content')[-1].text.strip()
            except:
                print(j)
                continue
            total.append([magazine_name[index],title, content, date[index].text.lstrip()[:10]])
        
        df_temp = pd.DataFrame(total, columns = ['신문사', '제목', '내용', '날짜'])
        df = pd.concat([df, df_temp], ignore_index = True)
    df.to_csv(f'{DateStart}_{DateEnd}.csv', encoding='utf-8')  

  0%|          | 0/556 [00:00<?, ?it/s]

84


TypeError: 'DatetimeIndex' object is not callable

In [4]:
import pandas as pd
news = pd.read_csv('2023-08-20_2023-08-27.csv')
news

,Unnamed: 0,신문사,제목,내용,날짜
0,0,세계일보,자영업자 신용대출 ‘저금리 대환’,코로나 기간내 고금리 대출자31일부터 최대 2000만원까지 신종 코로나바이러스 감염...,2023-08-27
1,1,경향신문,전세가구 이자비용 67% 급증… 월 20만원 첫 돌파,2023년 2분기 21만4000원 기록1년 반 만에 두 배 넘게 뛰어자가가구와 격차...,2023-08-27
2,2,경향신문,'잭슨홀 쇼크' 피했지만…'금리 딜레마' 여전한 한은,■우리 경제 여파는강도 낮췄지만 美 초긴축 여전유가상승 등으로 물가도 불안금리인하 ...,2023-08-27
3,3,세계일보,은행권 ‘부산 주 금고’ 잡기 ‘군침’… “지역 기여도 높여 평가해야”,지자체 등 70% 내년 계약 만료13곳 모두 부산은행 주 금고 거래시중은행 견제·도...,2023-08-27
4,4,서울경제,[단독] 슈퍼리치 놀이터라고?...10만원으로도 투자가능한 국채 나온다,서민 장기저축 지원 목적10년·20년 등 장기물로 출시만기보유시 분리과세·가산금리 ...,2023-08-27
...,...,...,...,...,...
2609,2609,연합뉴스,"중국발 경기 불안에…한은, 24일 기준금리 5연속 동결할 듯","전문가 ""2%p 한미 금리차·가계대출 증가에도 경기 우려로 못 올려"" ""한은, 올해...",2023-08-20
2610,2610,연합뉴스,"G2발 금융위기 확산…코스피 2,400까지 떨어지나","위안화·원화 약세에 외국인 매도압력…""환율 1,300원대 후반 가능성""""중국 부동산...",2023-08-20
2611,2611,YTN,문제 생기면 '은행 탓'…정부 가계빚 정책 엇박자 비판↑,금리 인하 옥죌 때는 언제고… 가계빚 늘자 또다시 은행권에 관리 주문50년 만기 주...,2023-08-20
2612,2612,노컷뉴스,"뉴욕증시, 엔비디아 실적 발표·연준 잭슨홀 컨퍼런스에 초점",[파이낸셜뉴스] 뉴욕증시 흐름을 좌우할 엔비디아 분기실적 발표가 23일(현지시간...,2023-08-20


In [10]:
import time
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

today = datetime.datetime.today()

DateStart = (today-datetime.timedelta(7)).strftime('%Y-%m-%d')
DateEnd = today.strftime('%Y-%m-%d')


last_url = f"https://finance.naver.com/news/news_search.naver?rcdate=&q=%B1%DD%B8%AE&x=20&y=14&sm=all.basic&pd=4&stDateStart={DateStart}&stDateEnd={DateEnd}&page=1"
headers = {'authority' : 'finance.naver.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
response = requests.get(last_url, headers=headers)
soup = BeautifulSoup(response.content)
news = soup.select('.pgRR a')[0]['href'][-3:]
news

'141'

In [ ]:
# 채권 기사
